# Tarefa de Programação: Otimization Methods

## Introdução

Bem-vindo à quarta tarefa (obrigatória) do Curso 2 da Especialização em Deep Learning! Neste notebook, você adquirirá habilidades com alguns métodos de otimização mais avançados que podem acelerar o aprendizado e talvez até mesmo levá-lo a um valor final melhor para a função de custo. Ter um bom algoritmo de otimização pode ser a diferença entre esperar dias e apenas algumas horas para obter um bom resultado.


# Optimization Methods

Until now, you've always used Gradient Descent to update the parameters and minimize the cost. In this notebook, you'll gain skills with some more advanced optimization methods that can speed up learning and perhaps even get you to a better final value for the cost function. Having a good optimization algorithm can be the difference between waiting days vs. just a few hours to get a good result. 

By the end of this notebook, you'll be able to: 

* Apply optimization methods such as (Stochastic) Gradient Descent, Momentum, RMSProp and Adam
* Use random minibatches to accelerate convergence and improve optimization

Gradient descent goes "downhill" on a cost function $J$. Think of it as trying to do this: 
<img src="images/cost.jpg" style="width:650px;height:300px;">
<caption><center> <u> <b>Figure 1</b> </u>: <b>Minimizing the cost is like finding the lowest point in a hilly landscape</b><br> At each step of the training, you update your parameters following a certain direction to try to get to the lowest possible point. </center></caption>

**Notations**: As usual, $\frac{\partial J}{\partial a } = $ `da` for any variable `a`.

Let's get started!

# Métodos de Otimização

Até agora, você sempre usou o Gradient Descent para atualizar os parâmetros e minimizar o custo. Neste notebook, você adquirirá habilidades com alguns métodos de otimização mais avançados que podem acelerar o aprendizado e talvez até mesmo levá-lo a um valor final melhor para a função de custo. Ter um bom algoritmo de otimização pode ser a diferença entre esperar dias e apenas algumas horas para obter um bom resultado.

Ao final deste caderno, você será capaz de:

* Aplicar métodos de otimização como (Estocástico) Gradient Descent, Momentum, RMSProp e Adam
* Use minilotes aleatórios para acelerar a convergência e melhorar a otimização

A descida do gradiente "desce" em uma função de custo $J$. Pense nisso como tentar fazer isso:
<img src="images/cost.jpg" style="width:650px;height:300px;">
<caption><center> <u> <b>Figura 1</b> </u>: <b>Minimizar o custo é como encontrar o ponto mais baixo em uma paisagem montanhosa</b><br> Em cada etapa de o treinamento, você atualiza seus parâmetros seguindo uma determinada direção para tentar chegar ao ponto mais baixo possível. </center></caption>

**Notações**: Como de costume, $\frac{\partial J}{\partial a } = $ `da` para qualquer variável `a`.

Vamos começar!

## Important Note on Submission to the AutoGrader

Before submitting your assignment to the AutoGrader, please make sure you are not doing the following:

1. You have not added any _extra_ `print` statement(s) in the assignment.
2. You have not added any _extra_ code cell(s) in the assignment.
3. You have not changed any of the function parameters.
4. You are not using any global variables inside your graded exercises. Unless specifically instructed to do so, please refrain from it and use the local variables instead.
5. You are not changing the assignment code where it is not required, like creating _extra_ variables.

If you do any of the following, you will get something like, `Grader not found` (or similarly unexpected) error upon submitting your assignment. Before asking for help/debugging the errors in your assignment, check for these first. If this is the case, and you don't remember the changes you have made, you can get a fresh copy of the assignment by following these [instructions](https://www.coursera.org/learn/deep-neural-network/supplement/QWEnZ/h-ow-to-refresh-your-workspace).

## 1- Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math
import sklearn
import sklearn.datasets

from opt_utils_v1a import load_params_and_grads, initialize_parameters, forward_propagation, backward_propagation
from opt_utils_v1a import compute_cost, predict, predict_dec, plot_decision_boundary, load_dataset
from copy import deepcopy
from testCases import *
from public_tests import *

%matplotlib inline
plt.rcParams['figure.figsize'] = (7.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

## 2 - Gradient Descent

A simple optimization method in machine learning is gradient descent (GD). When you take gradient steps with respect to all $m$ examples on each step, it is also called Batch Gradient Descent. 

### Exercise 1 - update_parameters_with_gd

Implement the gradient descent update rule. The  gradient descent rule is, for $l = 1, ..., L$: 
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{1}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{2}$$

where L is the number of layers and $\alpha$ is the learning rate. All parameters should be stored in the `parameters` dictionary. Note that the iterator `l` starts at 1 in the `for` loop as the first parameters are $W^{[1]}$ and $b^{[1]}$. 

## 2 - Descida Gradiente

Um método de otimização simples em aprendizado de máquina é o gradiente descendente (GD). Quando você executa etapas de gradiente em relação a todos os exemplos de $m$ em cada etapa, isso também é chamado de descida de gradiente em lote.

<a name='ex-1'></a>
### Exercício 1 - update_parameters_with_gd

Implemente a regra de atualização de gradiente descendente. A regra do gradiente descendente é, para $l = 1, ..., L$:
$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{1}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{2}$$

onde L é o número de camadas e $\alpha$ é a taxa de aprendizado. Todos os parâmetros devem ser armazenados no dicionário `parameters`. Observe que o iterador `l` começa em 1 no loop `for`, pois os primeiros parâmetros são $W^{[1]}$ e $b^{[1]}$.

In [6]:
# GRADED FUNCTION: update_parameters_with_gd

def update_parameters_with_gd(parameters, grads, learning_rate):
    """
    Update parameters using one step of gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters to be updated:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients to update each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    learning_rate -- the learning rate, scalar.
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    L = len(parameters) // 2 # number of layers in the neural networks

    # Update rule for each parameter
    for l in range(1, L + 1):
        # (approx. 2 lines)
        # parameters["W" + str(l)] =  
        # parameters["b" + str(l)] = 
        # YOUR CODE STARTS HERE
        parameters["W" + str(l)] = parameters["W" + str(l)] - learning_rate*grads["dW" + str(l)]  
        parameters["b" + str(l)] = parameters["b" + str(l)] - learning_rate*grads["db" + str(l)]
        
        # YOUR CODE ENDS HERE
    return parameters

In [7]:
parameters, grads, learning_rate = update_parameters_with_gd_test_case()
learning_rate = 0.01
parameters = update_parameters_with_gd(parameters, grads, learning_rate)

print("W1 =\n" + str(parameters["W1"]))
print("b1 =\n" + str(parameters["b1"]))
print("W2 =\n" + str(parameters["W2"]))
print("b2 =\n" + str(parameters["b2"]))

update_parameters_with_gd_test(update_parameters_with_gd)

W1 =
[[ 1.63535156 -0.62320365 -0.53718766]
 [-1.07799357  0.85639907 -2.29470142]]
b1 =
[[ 1.74604067]
 [-0.75184921]]
W2 =
[[ 0.32171798 -0.25467393  1.46902454]
 [-2.05617317 -0.31554548 -0.3756023 ]
 [ 1.1404819  -1.09976462 -0.1612551 ]]
b2 =
[[-0.88020257]
 [ 0.02561572]
 [ 0.57539477]]
All test passed


A variant of this is Stochastic Gradient Descent (SGD), which is equivalent to mini-batch gradient descent, where each mini-batch has just 1 example. The update rule that you have just implemented does not change. What changes is that you would be computing gradients on just one training example at a time, rather than on the whole training set. The code examples below illustrate the difference between stochastic gradient descent and (batch) gradient descent. 

- **(Batch) Gradient Descent**:

``` python
X = data_input
Y = labels
m = X.shape[1]  # Number of training examples
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    # Forward propagation
    a, caches = forward_propagation(X, parameters)
    # Compute cost
    cost_total = compute_cost(a, Y)  # Cost for m training examples
    # Backward propagation
    grads = backward_propagation(a, caches, parameters)
    # Update parameters
    parameters = update_parameters(parameters, grads)
    # Compute average cost
    cost_avg = cost_total / m
        
```

- **Stochastic Gradient Descent**:

```python
X = data_input
Y = labels
m = X.shape[1]  # Number of training examples
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):
    cost_total = 0
    for j in range(0, m):
        # Forward propagation
        a, caches = forward_propagation(X[:,j], parameters)
        # Compute cost
        cost_total += compute_cost(a, Y[:,j])  # Cost for one training example
        # Backward propagation
        grads = backward_propagation(a, caches, parameters)
        # Update parameters
        parameters = update_parameters(parameters, grads)
    # Compute average cost
    cost_avg = cost_total / m
```


Uma variante disso é a Descida de Gradiente Estocástico (SGD), que é equivalente à descida de gradiente de minilote, onde cada minilote tem apenas 1 exemplo. A regra de atualização que você acabou de implementar não muda. O que muda é que você estaria computando gradientes em apenas um exemplo de treinamento por vez, em vez de em todo o conjunto de treinamento. Os exemplos de código abaixo ilustram a diferença entre a descida de gradiente estocástica e a descida de gradiente (lote).

- **(Lote) Descida Gradiente**:

``` python
X = data_input
Y = rótulos
m = X.shape[1] # Número de exemplos de treinamento
parâmetros = initialize_parameters(layers_dims)
para i no intervalo (0, num_iterations):
     # Propagação para frente
     a, caches = forward_propagation(X, parâmetros)
     # Custo de cálculo
     cost_total = compute_cost(a, Y) # Custo para m exemplos de treinamento
     # Propagação para trás
     grads = reverse_propagation(a, caches, parâmetros)
     # Atualizar parâmetros
     parâmetros = update_parameters(parâmetros, graduados)
     # Calcule o custo médio
     custo_médio = custo_total / m
        
```

- **Descida do Gradiente Estocástico**:

```python
X = data_input
Y = rótulos
m = X.shape[1] # Número de exemplos de treinamento
parâmetros = initialize_parameters(layers_dims)
para i no intervalo (0, num_iterations):
     custo_total = 0
     para j no intervalo(0, m):
         # Propagação para frente
         a, caches = forward_propagation(X[:,j], parâmetros)
         # Custo de cálculo
         cost_total += compute_cost(a, Y[:,j]) # Custo para um exemplo de treinamento
         # Propagação para trás
         grads = reverse_propagation(a, caches, parâmetros)
         # Atualizar parâmetros
         parâmetros = update_parameters(parâmetros, graduados)
     # Calcule o custo médio
     custo_médio = custo_total / m
```

In Stochastic Gradient Descent, you use only 1 training example before updating the gradients. When the training set is large, SGD can be faster. But the parameters will "oscillate" toward the minimum rather than converge smoothly. Here's what that looks like: 

<img src="images/kiank_sgd.png" style="width:750px;height:250px;">
<caption><center> <u> <font color='purple'> <b>Figure 1</b> </u><font color='purple'>  : <b>SGD vs GD</b><br> "+" denotes a minimum of the cost. SGD leads to many oscillations to reach convergence, but each step is a lot faster to compute for SGD than it is for GD, as it uses only one training example (vs. the whole batch for GD). </center></caption>

**Note** also that implementing SGD requires 3 for-loops in total:
1. Over the number of iterations
2. Over the $m$ training examples
3. Over the layers (to update all parameters, from $(W^{[1]},b^{[1]})$ to $(W^{[L]},b^{[L]})$)

In practice, you'll often get faster results if you don't use the entire training set, or just one training example, to perform each update. Mini-batch gradient descent uses an intermediate number of examples for each step. With mini-batch gradient descent, you loop over the mini-batches instead of looping over individual training examples.

<img src="images/kiank_minibatch.png" style="width:750px;height:250px;">
<caption><center> <u> <font color='purple'> <b>Figure 2</b> </u>: <font color='purple'>  <b>SGD vs Mini-Batch GD</b><br> "+" denotes a minimum of the cost. Using mini-batches in your optimization algorithm often leads to faster optimization. </center></caption>

Em Stochastic Gradient Descent, você usa apenas 1 exemplo de treinamento antes de atualizar os gradientes. Quando o conjunto de treinamento é grande, o SGD pode ser mais rápido. Mas os parâmetros irão "oscilar" em direção ao mínimo em vez de convergir suavemente. Aqui está o que parece:

<img src="images/kiank_sgd.png" style="width:750px;height:250px;">
<caption><center> <u> <font color='purple'> <b>Figura 1</b> </u><font color='purple'> : <b>SGD vs GD</b><br> "+" denota um custo mínimo. O SGD leva a muitas oscilações para alcançar a convergência, mas cada etapa é muito mais rápida de calcular para SGD do que para GD, pois usa apenas um exemplo de treinamento (vs. todo o lote para GD). </center></caption>

**Observe** também que a implementação do SGD requer 3 for-loops no total:
1. Sobre o número de iterações
2. Sobre os $m$ exemplos de treinamento
3. Sobre as camadas (para atualizar todos os parâmetros, de $(W^{[1]},b^{[1]})$ a $(W^{[L]},b^{[L]}) $)

Na prática, você geralmente obterá resultados mais rápidos se não usar todo o conjunto de treinamento ou apenas um exemplo de treinamento para realizar cada atualização. A descida de gradiente em minilote usa um número intermediário de exemplos para cada etapa. Com a descida de gradiente de minilote, você percorre os minilotes em vez de percorrer exemplos de treinamento individuais.

<img src="images/kiank_minibatch.png" style="width:750px;height:250px;">
<caption><center> <u> <font color='purple'> <b>Figura 2</b> </u>: <font color='purple'> <b>SGD vs Mini-Batch GD</b><br> "+" denota um custo mínimo. O uso de mini-lotes em seu algoritmo de otimização geralmente leva a uma otimização mais rápida. </center></caption>